In [1]:
# Parameters
try:
    mode
except NameError:
    mode = None

try:
    input_path
except NameError:
    input_path = None

try:
    output_path
except NameError:
    output_path = None



In [2]:
# Parameters
mode = "fake"
input_path = ["C:\\store\\git\\km-stat-activity\\parquet_dataset\\date=2025-04-22\\profile_guid=cab111cc-6da4-4eab-b176-b2a611e6f157\\part.7.parquet"]
output_path = "C:\\store\\git\\km-stat-activity\\processed\\fake\\profile_guid=cab111cc-6da4-4eab-b176-b2a611e6f157\\2025-04-22-processed.csv"


In [3]:

import os
import pandas as pd
from pathlib import Path
import ast

# output_path tanımlı olmalı
output_path = str(output_path)

# Eğer df daha önce tanımlandıysa tekrar yükleme
if "df" not in globals():
    if isinstance(input_path, list):
        paths = [Path(p) for p in input_path]
        df = pd.concat([pd.read_parquet(p, engine="pyarrow") for p in paths], ignore_index=True)
    else:
        df = pd.read_csv(str(input_path),
                         converters={"x": pd.eval, "y": pd.eval},
                         parse_dates=['start_date_time', 'end_date_time'])
# x ve y listelerinin güvenli parse edilmesi (string ise)
def safe_parse_list(s):
    try:
        parsed = ast.literal_eval(s)
        return list(parsed)
    except Exception:
        return []

df['x'] = df['x'].apply(safe_parse_list)
df['y'] = df['y'].apply(safe_parse_list)

# Yön değişimlerini hesapla
def count_direction_changes(arr):
    count = 0
    for i in range(2, len(arr)):
        diff1 = arr[i-1] - arr[i-2]
        diff2 = arr[i] - arr[i-1]
        if diff1 * diff2 < 0:
            count += 1
    return count

df["x_direction_changes"] = df["x"].apply(count_direction_changes)
df["y_direction_changes"] = df["y"].apply(count_direction_changes)

# df'nin tamamını çıktı olarak al
df_result = df

# .1, .2 gibi tekrar sütunları engelle
df_result = df_result.loc[:, ~df_result.columns.str.contains(r'\.\d+$')]


# CSV’ye yaz (mevcutsa sütun olarak ekle)
if os.path.exists(output_path):
    df_existing = pd.read_csv(output_path)
    df_merged = pd.concat(
        [df_existing.reset_index(drop=True), df_result.reset_index(drop=True)],
        axis=1
    )
else:
    df_merged = df_result

df_merged.to_csv(output_path, index=False)
print(f"✔️ Saved CSV with full df including direction changes: {output_path}")


✔️ Saved CSV with full df including direction changes: C:\store\git\km-stat-activity\processed\fake\profile_guid=cab111cc-6da4-4eab-b176-b2a611e6f157\2025-04-22-processed.csv
